# Master Project, Notebook 1: data collection

In this notebook the necessary data for this project will be gathered.
On the one hand that means the data about the parties received funding and on the other hand the twitter data for the respective parties.

Based on these two dataset one then can decide on a timeframe and party sample in the 2nd script where then also more indeph financial data will be added.

In [82]:
############ packages

# general
from tqdm import tqdm
import numpy as np
import pandas as pd

import pickle

# 1.1 parsing with tabula
import tabula

# 1.2 correct parsed tables
import re

# 2 twitter scrape
import os
from datetime import date
import snscrape
import snscrape.modules.twitter as twitterScraper


## 1 data about party funding

The reports regarding the statefunding are available from 2002 onwards on the website of thee Bundestag (https://www.bundestag.de/parlament/praesidium/parteienfinanzierung/festsetz_staatl_mittel). Nevertheless, that availavility is restricted to pdf files. Since it is only 18 initial files a webscraping solution would have taken an unneccessary amount of time, I therefore downloaded them manually. 

Visually the tables contained in them seem to be standarized. After trying out differnt packages that promise the parsing of tables in pdf files to panda DataFrames, the package "tabula" has proven to be the one working best on the respective files. But the visual impression has prooven to be wrong - the parsed tables where different and false in a lot of ways. An extensive correction within python is necessary.



### 1.1 parsing with tabula

In [39]:
pdf_directory = "festsetzung_staatlicher_mittel_reports/"

filendings = ["0"+i if len(i)==1 else i for i in [str(i) for i in range(2,21)]]

tables_dict = {}

for i in tqdm(filendings):
    file = f"{pdf_directory}finanz_{i}-data.pdf"
    tables_dict[int(i)] = tabula.read_pdf(file, pages="all", multiple_tables=True)
    
print(list(tables_dict.items())[0])

 37%|██████████████████████████████▌                                                    | 7/19 [00:24<00:43,  3.62s/it]Got stderr: Dez. 17, 2021 10:15:26 VORM. org.apache.pdfbox.pdmodel.font.PDFontFactory createFont
WARNUNG: Invalid font subtype 'COSName{}'
Dez. 17, 2021 10:15:26 VORM. org.apache.pdfbox.pdmodel.font.PDType1Font <init>
WARNUNG: Using fallback font ArialMT for MS Sans Serif

 42%|██████████████████████████████████▉                                                | 8/19 [00:28<00:39,  3.64s/it]Got stderr: Dez. 17, 2021 10:15:30 VORM. org.apache.pdfbox.pdmodel.font.PDFontFactory createFont
WARNUNG: Invalid font subtype 'COSName{}'
Dez. 17, 2021 10:15:30 VORM. org.apache.pdfbox.pdmodel.font.PDType1Font <init>
WARNUNG: Using fallback font ArialMT for MS Sans Serif

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:31<00:00,  4.82s/it]

(2, [   Partei Bundestagswahl\r22.09.2002 Europawahl\r13.06.1999  \
0     NaN                Absolutin %            Absolutin %   

  Baden-Württemberg\r25.03.2001 Bayern\r13.09.1998 Berlin\r21.10.2001  \
0                   Absolutin %        Absolutin %        Absolutin %   

  Brandenburg\r05.09.1999 Bremen\r06.06.1999 Hamburg\r23.09.2001  \
0             Absolutin %        Absolutin %         Absolutin %   

  Hessen\r07.02.1999 Mecklenburg-Vorpommern\r22.09.2002  
0        Absolutin %                        Absolutin %  ,                      SPD  18.488.66838,52%   8.307.08530,70%  1.508.35833,29%  \
0                    CDU  14.167.56129,52%  10.628.22439,28%  2.029.80644,80%   
1                    CSU    4.315.0808,99%    2.540.0079,39%              NaN   
2                  GRÜNE    4.110.3558,56%    1.741.4946,44%     350.3837,73%   
3                    FDP    3.538.8157,37%      820.3713,03%     367.5808,11%   
4                    PDS    1.916.7023,99%    1.567.7455,79%  

### 1.2 correct parsed tables

Since the parsing ends up in severe irregularities and most of the data we might not need foor this project, i will for now only finalize the parsing for the last table that contains which parties receive what support to there Bundesverband (excluding the payments for the Landesverbände) since we initially want to look only at the Bundesverbände, later in the project we can expand that if needed.

The reports contain several tables where only the table on the last page was  the one I am interested in. Since the reports are differently parsed and have different page numbers it was not possible to only extract that one table.


In [55]:
# through manual checking we know at which position the right table is, depending on the year

# we get these ones into one dictionary
table_dict = {k:(v[-1] if k not in [17,18,19] else v[-2]) for (k,v) in tables_dict.items()}

# set up a vector for collumn names for the standartized table
col_titles = ["party","share_state","rest_prev_year","final_payment_prev_year",
             "rate_new_year"]

In [56]:
## group tables manually based on their issues to solve them efficiently

# tables 2-7,10,16 have the same problems
# 8 differently (title for last collumn moved one column to far)
# 9 the last two column names moved one column further
# 11 and 12 - last column moved one col to far
# 14 and 15 a lot of stuff 
# 17 and 18 a  lot of stuff 
# 19 mostly good
# 20 a lot stuff but IS last table


### set up helper funtions for corrections
def correct_table(table):
    '''
    corrects tables with cartain parsing errors:
    - 2 rows at the top to much, to many cols
    - all col values in one cell seperated with "/r"
    - adjust collumn names
    '''
    new_table = table.iloc[2:,:5]
    new_cols = {"party":new_table.iloc[:,0]}
    
    # loop through columns and set correct collumn titels and correct collumn content
    for i in range(1,5):
        new_cols[col_titles[i]] = new_table.iloc[0,i].split("\r")
        new_cols[col_titles[i]].append(new_table.iloc[-1,i])
    return pd.DataFrame(new_cols)


def correct_table_2020(table):
    '''
    adresses specifically the parsing errors of the 2020 table
    '''
    new_table = table.iloc[2:,:5]
    new_cols = {"party":new_table.iloc[:,0]}
    
    for i in range(1,5):
        new_col = new_table.iloc[0,i].split("\r")
        new_col.append(new_table.iloc[-1,i])
        if i >= 3:
            new_cols[col_titles[i]]=new_col[:9]+[np.nan]+new_col[9:]
        else:
            new_cols[col_titles[i]] = new_col
    
    return pd.DataFrame(new_cols)

In [57]:
# fix issues, align table formats

table_dict[8] = table_dict[8].rename(columns={"Unnamed: 0": "rate_new_year"}).iloc[:,:-1]

table_dict[9] = table_dict[9].rename(columns={"Unnamed: 0": "final_payment_prev_year", 
                                              "Schlusszahlung/\rRückforderung für 2009": "rate_new_year"}).iloc[:,:-1]
table_dict[11] = table_dict[11].drop("Unnamed: 0", axis=1)
table_dict[12] = table_dict[12].drop("Unnamed: 0", axis=1)

table_dict[15] = table_dict[15].rename(columns={"Unnamed: 0":"party",
                              "Unnamed: 2":"share_state",
                              "Unnamed: 3": "rest_prev_year",
                              "Unnamed: 4": "final_payment_prev_year",
                              "Höhe der einzelnen":"rate_new_year"}).drop("Unnamed: 1", axis=1).iloc[9:,:].drop(21,axis=0)


# most complex issues - use functions
table_dict[14] = table_dict[14].drop("Unnamed: 1",axis=1)
table_dict[14] = correct_table(table_dict[14])

table_dict[17] = table_dict[17].drop(["Unnamed: 0", "Unnamed: 2"],axis=1)
table_dict[17] = correct_table(table_dict[17])

table_dict[18] = table_dict[18].drop(["Unnamed: 0", "Unnamed: 1"],axis=1)
table_dict[18] = correct_table(table_dict[18])

table_dict[20] = table_dict[20].drop(["Unnamed: 0", "Unnamed: 1"],axis=1)
table_dict[20] = correct_table_2020(table_dict[20])

In [65]:
# adjust 

# drop the first two rows in all tables where needed
table_dict_2 = {k:(v.drop([0,1],axis=0)if k in [2,3,4,5,6,7,8,9,10,11,12,13,16,19] else v) for k,v in table_dict.items()}
table_dict_2[20].head()

,party,share_state,rest_prev_year,final_payment_prev_year,rate_new_year
2,1)\rCDU,"48.850.162,71","42.127.455,16","6.722.707,55","2.677.000,00"
3,SPD,"49.879.078,91","51.212.981,44","- 1.333.902,53","12.469.769,73"
4,GRÜNE,"22.841.953,77","21.262.989,52","1.578.964,25","5.710.488,44"
5,AfD,"9.290.136,62","7.694.174,96","1.595.961,66","2.322.534,16"
6,FDP,"14.130.047,11","13.848.084,88","281.962,23","3.532.511,78"


In [66]:
# set correct titels for all tables
for k,v in table_dict_2.items():
    table_dict_2[k].set_axis(col_titles, axis=1, inplace=True)
    
    
# some party names seem to be still false e.g. "1)\rCDU" instead of CDU
# adjust party names

def clean_party_name(s):
    '''
    clean text input with regx expressions
    '''
    output = re.sub(r"\d*\)"," ",s)
    return output.strip(" ").strip("\r")

# apply cleaning function
for k,v in table_dict_2.items():
    table_dict_2[k].party = [clean_party_name(c) for c in table_dict_2[k].party]

In [72]:
# adjust party names further (different names for similar parties)


party_names = {"Die Tierschutzpartei": "Tierschutzpartei",
              "ödp":"ÖDP",
               "Die Linke.":"DIE LINKE",
               "Die PARTEI":"DIE PARTEI",
               "BVB/Freie Wähler":"BVB/FREIE WÄHLER",
               "AFD":"AfD",
               "FW FREIE WÄHLER":"FREIE WÄHLER",
               "FW-SH":"FW Schleswig-Holstein"
              }

for k,_ in table_dict_2.items():
    new_col = []
    for p in list(table_dict_2[k].partei):
        if p in party_names.keys():
            new_col.append(party_names[p])
        else:
            new_col.append(p)
    table_dict_2[k].partei = new_col

In [76]:
# set up year column for each tale and concat all tables into one dataframe

table_dict_2[2]["jahr"] = 2002
data = table_dict_2[2]

for k,v in table_dict_2.items():
    if k != 2:
        table = v
        if len(str(k)) == 1:
            table["year"] = int("200"+str(k))
            data = pd.concat([data,table], ignore_index=True)
        else:
            table["year"] = int("20"+str(k))
            data = pd.concat([data,table], ignore_index=True)
        

In [78]:
# test
data[data.party == "PIRATEN"]

,party,share_state,rest_prev_year,final_payment_prev_year,rate_new_year,jahr,year
152,PIRATEN,"0,00","0,00","0,00","0,00",NaN,2009.0
177,PIRATEN,"492.895,46","0,00","492.895,46","123.223,87",NaN,2010.0
199,PIRATEN,"331.238,05","492.895,48","- 161.657,43","82.809,51",NaN,2011.0
222,PIRATEN,"243.580,67","331.238,04","- 87.657,37","60.895,17",NaN,2012.0
244,PIRATEN,"1.063.061,20","243.580,68","819.480,52","265.765,30",NaN,2013.0
267,PIRATEN,"939.344,98","1.063.061,20","- 123.716,22","234.836,25",NaN,2014.0
288,PIRATEN,"242.755,26","939.345,00","- 696.589,74","60.688,82",NaN,2015.0
309,PIRATEN,"263.664,35","242.755,28","20.909,07","65.916,09",NaN,2016.0
333,PIRATEN,"587.660,39","263.664,36","323.996,03","146.915,10",NaN,2017.0
355,PIRATEN,"526.712,53","587.660,40","- 60.947,87","131.678,13",NaN,2018.0


In [81]:
file = open("finanz_data.obj","wb")
pickle.dump(data,file)
file.close()

## 2 twitter data

The search for a package that allows the fetching tweets back to the first tweet ever published by a respective account took longer than expected. While the original twitter API does not promises the necessary features "twint" and "TwitterScraper" did so, but out of different reasons they could not fetch the needed tweets. I finally found "snscrape" that at least as far as I can judge works a I want it.

In [ ]:
### possible endpoints of the snscrape API
#   url: str
# 	date: datetime.datetime
# 	content: str
# 	renderedContent: str
# 	id: int
# 	user: 'User'
# 	replyCount: int
# 	retweetCount: int
# 	likeCount: int
# 	quoteCount: int
# 	conversationId: int
# 	lang: str
# 	source: str
# 	sourceUrl: typing.Optional[str] = None
# 	sourceLabel: typing.Optional[str] = None
# 	outlinks: typing.Optional[typing.List[str]] = None
# 	tcooutlinks: typing.Optional[typing.List[str]] = None
# 	media: typing.Optional[typing.List['Medium']] = None
# 	retweetedTweet: typing.Optional['Tweet'] = None
# 	quotedTweet: typing.Optional['Tweet'] = None
# 	inReplyToTweetId: typing.Optional[int] = None
# 	inReplyToUser: typing.Optional['User'] = None
# 	mentionedUsers: typing.Optional[typing.List['User']] = None
# 	coordinates: typing.Optional['Coordinates'] = None
# 	place: typing.Optional['Place'] = None
# 	hashtags: typing.Optional[typing.List[str]] = None
# 	cashtags: typing.Optional[typing.List[str]] = None

In [83]:
# define helper function to scrape all tweets of a respective user

def scrape_user_to_df(user, col):
    scraper = twitterScraper.TwitterUserScraper(user, False)
    out_df = pd.DataFrame(columns=col)

    for i,tweet in enumerate(scraper.get_items()):
        row = [tweet.date, tweet.content, tweet.renderedContent,tweet.id, tweet.user,tweet.replyCount,
              tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.mentionedUsers, tweet.hashtags,
              tweet.cashtags, tweet.media, tweet.outlinks, tweet.retweetedTweet,tweet.quotedTweet, tweet.inReplyToTweetId]
        out_df.loc[i] = row
        if i%1000 == 0:
            print(i, tweet.date)
    return out_df

# def function to scrape all tweets of multiple users and return a dataframe
def scrape_multiple_users(list_users):
    col = ["date","content","renderedContent", "id","user","replyCount",
          "retweetCount", "likeCount","quoteCount","mentionedUsers", "hashtags",
          "cashtags","media", "outlinks","retweetedTweet","quotedTweet","inReplyToTweetId"]
    out_df = pd.DataFrame(columns=col)
    for u in tqdm(list_users):
        df_temp = scrape_user_to_df(u, col)
        out_df = pd.concat([out_df,df_temp],ignore_index=True)
        
        print(f"{u} scraped.")
    return out_df

In [84]:
#  set up list of all found twitter accounts of parties that where never part of the Bundestag
party_list = ["FREIEWAEHLER_BV","DiePARTEI","Tierschutzparte","oedp_de","Piratenpartei","VoltDeutschland","Bayernpartei",
 "BVBFW","FREIEWAEHLER_BV","LKR_Partei_Bund","SSWpresse","npdde","TierschutzH","Allianz_MTN","republikaner",
 "freiesachsen_"]

In [86]:
data_complete = scrape_multiple_users(party_list)
# for an direct impression of what amount of data we can expect the function prints after every 1000 tweets the corresponding date


# depending on where one runs snscrape  one might get at some point the following error
## ScraperException: Unable to find guest token
## accoridng to the online community that error is caused by twitter blocking certain apis
# however I managed to run the full scriped through my home network, the needed data is therefore successfull scraped



  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

0 2021-12-12 09:00:02+00:00


  6%|█████▏                                                                             | 1/16 [00:24<06:05, 24.39s/it]

FREIEWAEHLER_BV scraped.
0 2021-12-06 14:00:02+00:00
1000 2019-04-17 11:20:42+00:00
2000 2010-11-24 15:23:10+00:00


 12%|██████████▍                                                                        | 2/16 [01:42<13:03, 55.96s/it]

DiePARTEI scraped.
0 2021-12-16 13:58:02+00:00
1000 2020-08-08 11:36:06+00:00
2000 2019-08-28 16:12:11+00:00
3000 2018-08-14 16:39:38+00:00
4000 2014-05-26 08:10:33+00:00
5000 2013-09-25 18:56:33+00:00
6000 2013-09-04 19:40:04+00:00
7000 2013-08-20 18:13:18+00:00


 19%|███████████████▍                                                                  | 3/16 [05:42<30:22, 140.16s/it]

Tierschutzparte scraped.


 19%|███████████████▍                                                                  | 3/16 [05:44<24:52, 114.80s/it]


ScraperException: Unable to find guest token

In [ ]:
data_complete.head() # check fetched data

In [ ]:
# save once fetched so we dont need to fetch the data again
# with open("twitter_parties.pickle", "wb") as f:
#     pickle.dump(data_complete, f)